<a href="https://colab.research.google.com/github/minsuk-heo/tf2/blob/master/jupyter_notebooks/07.CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN (Convolutional Neural Network)

In [1]:
from IPython.display import Image

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

tf.random.set_seed(1)

In [3]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Preprocessing

In [4]:
# collect MNIST data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
# sample to show gray scale values
print(x_train[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


In [6]:
# sample to show labels for first train data to 10th train data
print(y_train[0:9])

[5 0 4 1 9 2 1 3 1]


In [7]:
print("test data has " + str(x_test.shape[0]) + " samples")
print("every test data is " + str(x_test.shape[1]) 
      + " * " + str(x_test.shape[2]) + " image")

test data has 10000 samples
every test data is 28 * 28 image


In [8]:
# reshape data
import numpy as np
x_train = np.reshape(x_train, (60000,28,28,1))
x_test = np.reshape(x_test, (10000,28,28,1))

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [9]:
# normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

In [10]:
# change label to one hot encoding
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

### We will implement below CNN

In [11]:
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/practice_cnn.png", width=800, height=200)

In [12]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(28,28,1),padding='same'))  # (필터의 커널갯수, 커널사이즈, 활성함수, 인풋, 패딩)
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',padding='same'))  # same : 출력 이미지 사이즈가 입력 이미지 사이즈와 동일
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               200832    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [14]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [15]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=False),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)]

In [16]:
model.fit(x_train, y_train,
          batch_size=500,
          epochs=10,
          verbose=1,
          validation_split = 0.1, 
          callbacks=callbacks)

Epoch 1/10
108/108 [==============================] - 33s 23ms/step - loss: 0.4869 - accuracy: 0.8704 - val_loss: 0.1215 - val_accuracy: 0.9657
Epoch 2/10
108/108 [==============================] - 2s 21ms/step - loss: 0.1076 - accuracy: 0.9682 - val_loss: 0.0772 - val_accuracy: 0.9787
Epoch 3/10
108/108 [==============================] - 2s 22ms/step - loss: 0.0713 - accuracy: 0.9784 - val_loss: 0.0571 - val_accuracy: 0.9843
Epoch 4/10
108/108 [==============================] - 2s 21ms/step - loss: 0.0541 - accuracy: 0.9837 - val_loss: 0.0478 - val_accuracy: 0.9872
Epoch 5/10
108/108 [==============================] - 2s 21ms/step - loss: 0.0447 - accuracy: 0.9865 - val_loss: 0.0414 - val_accuracy: 0.9888
Epoch 6/10
108/108 [==============================] - 2s 21ms/step - loss: 0.0392 - accuracy: 0.9880 - val_loss: 0.0417 - val_accuracy: 0.9882
Epoch 7/10
108/108 [==============================] - 2s 21ms/step - loss: 0.0317 - accuracy: 0.9899 - val_loss: 0.0416 - val_accuracy: 0.988

In [17]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.032092515379190445
Test accuracy: 0.9894000291824341
